<a href="https://colab.research.google.com/github/KimRass/KimRass/blob/main/pozalabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount google drive

from google.colab import drive
import os

drive.mount("/content/drive")

In [ ]:
# Clone github repository

!mkdir github
os.chdir("github")
!git clone https://github.com/magenta/magenta.git

Cloning into 'magenta'...
remote: Enumerating objects: 15910, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 15910 (delta 24), reused 46 (delta 13), pack-reused 15847
Receiving objects: 100% (15910/15910), 36.43 MiB | 15.93 MiB/s, done.
Resolving deltas: 100% (12078/12078), done.


In [ ]:
# Install library 'magenta'

!pip install magenta

In [ ]:
# Import librarie(s)

import warnings

# 경고 메시지가 너무 많이 출력되어 이를 억제합니다.
warnings.filterwarnings("ignore")

In [ ]:
# Preprocess dataset

# 'Groove MIDI Dataset'의 .midi 파일들로부터 하나의 .tfrecord 파일을 생성합니다.
# 이를 위해서 'content/drive/MyDrive/groove'에 이 데이서셋을 압축을 풀어 저장합니다.
# 이 데이터셋은 드럼 연주를 녹음한 것입니다.
# 그러나 `tensorflow_datasets`로부터 손쉽게 이 데이터셋을 가지고 올 수 있으므로 꼭 필요한 과정은 아닙니다.
# # 4마디의 드럼 연주를 생성할 것이므로 'groove/4bar-midionly'를 사용합니다.
# (https://www.tensorflow.org/datasets/catalog/groove#groove4bar-midionly)
# 아래에서 학습에는 `tensorflow_datasets`를 사용할 것이므로 이 셀은 실행하지 않아도 됩니다.
dir_input="../drive/MyDrive/groove"
!mkdir dir_input

!python3 magenta/magenta/scripts/convert_dir_to_note_sequences.py\
    --input_dir="../drive/MyDrive/groove"\
    --output_file="../drive/MyDrive/groove_midi_dataset.tfrecord"\
    --recursive

In [ ]:
# Train

dir_tr="../drive/MyDrive/train_groovae_4bar"
!mkdir dir_tr

# `--config`:
    # 4마디의 드럼 연주를 생성할 것이므로 'groovae_4bar'를 사용합니다.
    # 'GrooVAE'를 'MusicVAE'의 변형체로서 드럼 연주를 생성하기 위한 것이라고 합니다.
    # (https://github.com/magenta/magenta/tree/main/magenta/models/music_vae)
!python3 ./magenta/magenta/models/music_vae/music_vae_train.py\
    --config="groovae_4bar"\
    --run_dir=$dir_tr\
    --mode="train"\
    --tfds_name="groove/4bar-midionly"\
    --cache_dataset=True\
    --hparams=batch_size=128

# 시간이 꽤 오래 걸리므로 Loss가 적당히 낮아질 때까지만 학습을 시켰습니다.
# Loss가 초기 565 정도에서 66정도까지 감소했습니다.
# 학습이 중단된 후 재개됐을 경우 마지막 Checkpoint부터 다시 할습시킬 수 있습니다.

/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

In [ ]:
# Generate .midi files by sampling

dir_output="../drive/MyDrive/generated"
!mkdir $dir_output

# `--checkpoint_file`: `tf.train.latest_checkpoint()`을 사용하여 마지막의 Checkpoint를 자동으로 사용합니다.
!python3 ./magenta/magenta/models/music_vae/music_vae_generate.py\
    --config="groovae_4bar"\
    --checkpoint_file="../drive/MyDrive/train_groovae_4bar/train"\
    --mode="sample"\
    --num_outputs=5\
    --output_dir=$dir_output

# 생성된 파일을 청음해보니 꽤나 자연스럽긴 하지만 부분적으로 소리가 작아지거나 커지는 경우가 있었습니다.

mkdir: cannot create directory ‘../drive/MyDrive/generated’: File exists
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_IN

In [ ]:
# Generate .midi files by interpolation

# 두 개의 .midi 파일을 Representation space 상에 위치시켰을 때, 두 점을 잇는 직선을 (n + 1)등분하는 n개의 점을 찾는 방식입니다.
# 위에서 'sample' 모드로 생성한 파일 중 2개를 사용해보겠습니다.

dir_output="../drive/MyDrive/generated"
!mkdir $dir_output

# `--input_midi_1`, `--input_midi_2`:
    # Interpolation에 사용할 .midi 파일은 사용할 모델에 적잡한 마디 수를 가져야 하고
    # 사용된 악기도 모델과 일치해야 합니다.
!python3 ./magenta/magenta/models/music_vae/music_vae_generate.py\
    --config="groovae_4bar"\
    --checkpoint_file="../drive/MyDrive/train_groovae_4bar/train"\
    --mode="interpolate"\
    --num_outputs=3\
    --input_midi_1="../drive/MyDrive/generated/groovae_4bar_sample_2022-12-18_043044-000-of-005.mid"\
    --input_midi_2="../drive/MyDrive/generated/groovae_4bar_sample_2022-12-18_043044-001-of-005.mid"\
    --output_dir=$dir_output

mkdir: cannot create directory ‘../drive/MyDrive/generated’: File exists
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_IN